<img src="Maslow's_Hierarchy_of_Needs.svg.png" height = 600px>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import urllib.parse # combine URL components into URL string
import wikipediaapi # query wikipedia through api
import requests # standard for making HTTP requests

from statsmodels.tsa.seasonal import STL # seasonal decompositions
import statsmodels.tsa.stattools as smt


import pickle #  to serialize and deserialize objects in Python

from scipy import signal
import warnings
from urllib import request
from bs4 import BeautifulSoup
import json
import pytrends
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import wikipediaapi
import pprint as pp

In [2]:
# 
def create_dataframe(name_file):
    """
    Creates a dataframe from a text file
    param: name_file: name of the text file
    return: dataframe with the text file
    """
    df = pd.read_csv(name_file, header=None, names=['Topics'])
    df.drop_duplicates(inplace=True)
    return df

# Take only starting from the second word in each row
def remove_space(df):
    return df['Topics'].apply(lambda x: x.strip().replace(' ', '_'))

# Parse the topics into the URL format
def parse_topics_into_df(df, start_time, end_time):
    # change the spaces to underscores
    df['url'] = np.zeros(len(df))
    for index, row in df.iterrows():
        topic_value = row['Topics']
        df.loc[index, 'url'] = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/{topic_value}/monthly/{start_time}/{end_time}'

    return df

# Create a new dataframe with timestamp from starting date to ending date
def create_dataframe_timestamp(starting_date, ending_date):
    df_timestamp = pd.DataFrame()
    df_timestamp['Timestamp'] = pd.date_range(start=starting_date, end=ending_date, freq='MS')
    return df_timestamp

# Define a function to fetch data from the URL and handle errors
def fetch_and_parse_url(url):
    try:
        request.urlopen(url).read()
        return True
    except request.HTTPError as e:
        if e.code == 404:
            return None  # or any other value or action you prefer for 404 errors
        else:
            return None  # or handle other HTTP errors as needed
    except Exception as e:
        
        return None  # or handle other exceptions as needed

def get_pageviews_wiki(url):
    """
    Gets the weekly pageviews for one Wikipedia page in one language in the desired period
    param: url: url of the Wikipedia page
    param: start_date: beginning of the desired period 
    param: end_date: end of the desired period 
    return: dataframe column with the monthly pageviews
    """
    html = request.urlopen(url).read()
    soup = BeautifulSoup(html,'html.parser')
    site_json=json.loads(soup.text)
    df=pd.DataFrame(site_json['items'])
    df=df['views']
    return df

def scrape_pageviews(df):
    pageview = pd.DataFrame()
    pageview['Timestamp'] = create_dataframe_timestamp('2019-10-01', '2023-02-01')['Timestamp']

    # Loop through the rows of the DataFrame and append the results of the function to the DataFrame
    for index, row in df.iterrows():
        url = row['url']
        if fetch_and_parse_url(url):
            pageview[row['Topics']] = get_pageviews_wiki(url)

    return pageview

# Specify the path to your text file
text_file_path = 'morality.txt'

def create_dataframe_pageviews(name_file, start_time = '20191001', end_time = '20230213'):
    """
    Creates a dataframe from a text file
    param: name_file: name of the text file, start_time: beginning of the desired period, end_time: end of the desired period
    
    """
    df_topic = create_dataframe(name_file)
    df_topic['Topics'] = remove_space(df_topic)
    df_topic = parse_topics_into_df(df_topic, start_time, end_time)
    df_pageviews = scrape_pageviews(df_topic)
    return df_pageviews

# Read the text file into a DataFrame
df_pageviews = create_dataframe_pageviews(text_file_path)
df_pageviews

,Timestamp,Moral_psychology,Global_ethics,Animal_ethics,Medical_ethics,Criminal_justice_ethics,Ethics_in_journalism,Moral_intuition,Moral_reasoning,Moral_responsibility
0,2019-10-01,5225,111,2091,29150,1725,29,46,4738,10537
1,2019-11-01,5545,122,2732,27906,1800,19,67,4325,10605
2,2019-12-01,5649,91,2027,23737,1700,20,46,3798,9256
3,2020-01-01,5088,101,2297,23938,1539,22,59,3781,9712
4,2020-02-01,4513,70,1984,22220,1405,10,39,4154,8542
5,2020-03-01,4295,76,2104,25564,1118,18,38,3759,7930
6,2020-04-01,5814,92,2509,24651,1778,28,45,3403,7938
7,2020-05-01,4601,120,1868,20971,1129,32,52,3590,9013
8,2020-06-01,3638,80,1975,20025,1458,22,54,3476,7197
9,2020-07-01,3843,87,2308,17011,810,15,47,2839,5693
